In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random


In [10]:
# dataset1 = pd.read_csv("data1.txt", delimiter = "\t", skiprows = [0,0])
dataset2 = pd.read_csv("data2.txt", delimiter = "\t")
dataset3 = pd.read_csv("data3.txt", delimiter = "\t")

In [11]:
dataset1 = pd.read_csv("data1.txt", delim_whitespace=True, dtype=str) # 32 rows x 5 variables (+ class, space separated, CR EOL)

In [12]:
dataset1.loc[0][0]

'00000'

In [13]:
np_data_set = []


for item in dataset1.itertuples():
    bit_string = str(item[1])
    output_string = str(item[2])
    np_bit_string = np.fromstring(bit_string,'u1') - ord('0')
    np_output_string = np.fromstring(output_string,'u1') - ord('0')
    # print(np_bit_string, np_output_string)
    np_data_set.append([np_bit_string, np_output_string])

<ipython-input-13-8fabb802a3bc>:7: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  np_bit_string = np.fromstring(bit_string,'u1') - ord('0')
<ipython-input-13-8fabb802a3bc>:8: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  np_output_string = np.fromstring(output_string,'u1') - ord('0')


In [680]:
def schema_model(schema_bitstring_array: 'np.ndarray', schema_target: 'np.ndarray', test_data_array: 'np.ndarray', test_target_result: 'np.ndarray') -> bool:
    """
    This function handles np.ndarray of 1D, lists, that is to say.
    : schema is the mask
    : test is data from the dataset
    """

    # we take the schmea (the values that are 1), and take values as per it from the test_data_array
    # 0 to 5 is the schema (the 5th index is actually the 6th value, but numpy doesnt touch the last value. but only the value before that)
    # ie: we use the schmea as a mask/ filter over the test data and extract those values (which are zeros or ones)

    # print(schema_bitstring_array, "schema_bitstring_array")
    # print(schema_bitstring_array[0:5], "schema_bitstring_array[0:5]")
    # print(test_data_array, "test_data_array")
    data_array = test_data_array[schema_bitstring_array == 1]
    # print(data_array, "filtered data")
    
    # we use the sum the filtered data into an integer
    sum_of_data_array = data_array.sum()
    # print(sum_of_data_array)

    # this is hypothesized output from the schema
    schema_target_output = sum_of_data_array >= schema_target[0]

    # compare against the actual output
    actual_output = None 
    
    if test_target_result[0] == 0:
        actual_output = False
    elif test_target_result[0] == 1:
        actual_output = True
    
    does_both_match = actual_output and schema_target_output

    # print(actual_output, schema_target_output, does_both_match)
    return does_both_match



# first 5 values have to be bool values, the 6th value has to be a value between 1 and 5
random_matrix_array = np.random.randint(2,size=(1,5))
random_matrix_array_2 = np.random.randint(2,size=(1,5))
random_target = np.random.choice([0, 1], size=(1, 1)) # actual target output
random_target_t = np.random.choice([0, 1, 2, 3, 4], size=(1, 1)) # hypothesized random output // np_data_set


# print("random schema", random_matrix_array[0], "random sum threhold", random_target_t[0])
# print("actual data  ", random_matrix_array_2[0], "output             ", random_target[0])
# schema_model(schema_bitstring_array=random_matrix_array[0], schema_target=random_target_t[0], test_data_array=random_matrix_array_2[0], test_target_result=random_target[0])

random_int = random.randrange(0, 30, 1)

# print("random schema", random_matrix_array[0], "random sum threhold", random_target_t[0])
# print("actual data  ", np_data_set[random_int][0], "output             ", np_data_set[random_int][1])

schema_model(schema_bitstring_array=random_matrix_array[0], schema_target=random_target_t[0], test_data_array=np_data_set[random_int][0], test_target_result=np_data_set[random_int][1])

False

In [1181]:
def schema_model_v2(schema_bitstring_array: 'np.ndarray', schema_target: 'np.ndarray', schema_value_map: 'np.ndarray', test_data_array: 'np.ndarray', test_target_result: 'np.ndarray') -> bool:
    """
    This function handles np.ndarray of 1D, lists, that is to say.
    : schema is the mask
    : test is data from the dataset
    """
    # print(test_data_array, " ", schema_value_map)

    data_array = test_data_array[schema_bitstring_array == 1]
    value_array = schema_value_map[schema_bitstring_array == 1]
    mult_array = np.matmul(data_array, value_array)

    # print(data_array, "filtered data")
    
    # we use the sum the filtered data into an integer
    sum_of_data_array = mult_array.sum()
    # print(sum_of_data_array)

    # this is hypothesized output from the schema
    schema_target_output = sum_of_data_array >= schema_target[0]

    # compare against the actual output
    actual_output = None 
    
    if test_target_result[0] == 0:
        actual_output = False
    elif test_target_result[0] == 1:
        actual_output = True
    
    does_both_match = actual_output and schema_target_output

    # print(actual_output, schema_target_output, does_both_match)
    return does_both_match

#####################################################################

class Chromosome:
    def __init__(self):
        # the schema for fitness function is the genome, it is an array of [[Bool, Bool, Bool, Bool, Bool], [1 to 5]]
        self.dna_schema = []
        self.dna_schema_value_map = []
        self.dna_target = []
        self.fitness = 0.0

    def initialize(self):
        random_matrix_array = np.random.randint(2,size=(1,5))
        self.dna_schema = random_matrix_array[0]


        random_matrix_array_2 = np.random.randint(100,size=(1,5))
        self.dna_schema_value_map = random_matrix_array_2[0]

        # the schema function works against numpy arrays, so just wrap the target in one too
        # random_target  = np.random.choice([0, 1, 2, 3, 4, 5], size=(1, 1))

        max_target = random.randint(1, random_matrix_array_2.sum())
        np_random_target_2 = np.array([max_target])
        self.dna_target = np_random_target_2

        # random_target = random_matrix_array.sum()
        # np_random_target = np.array([random_target])
        # self.dna_target = np_random_target
        # print(self.dna_target)

    def calculate_fitness(self, actual_dataset, fitness_function):
        # actual_data_string = actual_dataset[1][0]
        # actual_result = actual_dataset[1][1]

        # is_match = fitness_function(schema_bitstring_array=self.dna_schema, schema_target=self.dna_target, test_data_array=actual_data_string, test_target_result=actual_result)
        # print(is_match)


        for actual_data_string, actual_result in actual_dataset:
            is_match = fitness_function(schema_bitstring_array=self.dna_schema, 
                                        schema_target=self.dna_target, 
                                        schema_value_map=self.dna_schema_value_map, 
                                        test_data_array=actual_data_string, 
                                        test_target_result=actual_result)
            
            if is_match:
                self.fitness+=1

        print(self.fitness, self.dna_schema, self.dna_schema_value_map, self.dna_target)

    def cross_over(self):
        pass

    def mutate(self):
        pass

    def __str__(self):
        return f"schema: {self.dna_schema}, target: {self.dna_target}, value_map: { self.dna_schema_value_map }"

    
    def __repr__(self):
        return f"schema: {self.dna_schema}, target: {self.dna_target}, value_map: { self.dna_schema_value_map }"


for _ in range(0, 100):
    x = Chromosome()
    x.initialize()
    x.calculate_fitness(actual_dataset=np_data_set, fitness_function=schema_model_v2)

0.0 [1 1 0 0 1] [20 39 11 82 48] [139]
0.0 [0 0 1 1 0] [88 83 40 39 95] [250]
12.0 [0 0 1 1 1] [73 94 24 49 42] [37]
9.0 [1 0 1 1 1] [84 49 19 28 28] [65]
8.0 [1 0 1 0 1] [79 97 10 42 28] [39]
0.0 [0 0 0 0 0] [40 55 55 57 72] [39]
0.0 [0 1 1 0 0] [85 24 39 34 92] [118]
8.0 [0 1 1 1 0] [68 61 72 76 67] [116]
2.0 [1 1 0 0 1] [54 73 17 85 61] [168]
0.0 [1 1 0 1 1] [52  4 41 11 25] [113]
4.0 [1 0 0 1 1] [72  8 96  6 97] [158]
0.0 [1 1 1 0 0] [25 86 31 94 55] [166]
0.0 [0 1 0 0 0] [ 8 73 83 23 58] [221]
0.0 [1 1 0 1 1] [26 74 64 98 45] [245]
0.0 [1 1 0 0 1] [53 35 57 99 68] [228]
12.0 [1 1 0 0 0] [86 82 16  9 29] [34]
8.0 [0 0 1 0 1] [10  0 67 47 47] [55]
15.0 [1 0 1 1 1] [11 39  6 30  3] [3]
1.0 [1 0 1 1 1] [89 78 91 49 99] [316]
11.0 [0 1 1 1 1] [82 16 46 39 65] [60]
12.0 [1 0 1 0 0] [32 61 80 29 15] [19]
12.0 [0 1 0 0 1] [39 60 19 99 84] [30]
14.0 [0 1 1 1 1] [19 92 69  7 32] [28]
12.0 [1 0 1 0 1] [ 0 33 48 14 92] [20]
0.0 [0 1 0 0 1] [18  7 15 53 23] [52]
15.0 [1 1 1 1 0] [23 68 73 37  

0